In [2]:
import pandas as pd
import numpy 
import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

#use RAW descriptions, only removing stopwords
data = pd.read_csv("data/winemag-data-130k-v2.csv")
data.drop(columns=["Unnamed: 0"],inplace=True)
data = data.drop_duplicates(["description","price","points","taster_name","title"])
print(len(data))
data.drop(["taster_twitter_handle"],axis=1,inplace=True)
avgPriceWinery = data.groupby("winery")#.price.mean()
#avgPriceWinery.get_group("1+1=3").price.mean()
data['price'] = data.apply(
    lambda row: avgPriceWinery.get_group(row["winery"]).price.mean() if pd.isnull(row['price']) else row['price'],
    axis=1
)
data["price"].fillna(data["price"].mean(),inplace=True)

#prende solo recensioni con autore conosciuto, ha senso come cosa?
#data.dropna(subset=["taster_name"],inplace=True)

data.fillna("unknown",inplace=True)
data = data.groupby("country").filter(lambda x: len(x)>600) 
data = data.reset_index(drop=True)

In [6]:
data = pd.read_csv("Our_dataset/stemmed.csv")
#remove comment if you want to work with stemmed data
from nltk.corpus import stopwords
stop = stopwords.words('english')

data["word_count"] = data["description"].apply(lambda x: len(str(x).split(" ")))

data["stopwords"] = data["description"].apply(lambda x: len([x for x in x.split() if x in stop]))

In [ ]:
len(data)

117652

In [ ]:
sentences = []
for idx,row in data.iterrows():
    c = gensim.utils.simple_preprocess(row["description"])  #apply a bit of preprocessing done automatically
    sentences.append(c)

In [ ]:
model = Word2Vec(sentences,size=100,window=10,min_count=2,workers=5)
model.train(sentences, total_examples=len(sentences), epochs=10)

In [ ]:
len(model.wv.vocab)

In [ ]:
model.wv.vocab

In [ ]:
model.wv.most_similar(positive = ['lemon'])

In [ ]:
model.similarity("aroma","dirt")

In [ ]:
%%time
model.init_sims(replace=True)

In [ ]:
temp = data[data["variety"] == "Pinot Gris"]
print(temp.iloc[2])
print(temp.iloc[1])
print(sentences[91],sentences[13409])
distance = model.wmdistance(sentences[91], sentences[9])
print(distance)

In [ ]:
uniqueVars = data["variety"].unique().tolist()
uniqueVars

In [ ]:
%%time
canonDict = {}
for el in uniqueVars:
    temp = data[data["variety"] == el] 
    els = temp.nlargest(3,"points")       #find the 3 best rated to look at their descriptions
    temp1 = []
    for idx,row in els.iterrows():
        temp1.append(idx)
    canonDict[el] = temp1


In [ ]:
%%time
canonDict

In [ ]:
%%time
temp = []
for idx,row in data.iterrows():
    value = 0
    c = canonDict[data.iloc[idx]["variety"]] 
    for i in c:
        value = value + model.wmdistance(sentences[idx], sentences[i]) #use distance, so the lesser the distance, the more similar the values
    value = value/len(c)  #didn't use similarity because measure could have been negative
    print(idx)
    temp.append(value)

In [ ]:
data["similarityTop3WinesByVariety"] = temp 

In [ ]:
data.drop(["word_count","stopwords"],inplace=True,axis=1)

In [ ]:
data.to_csv("StemmedWord2vecTop3.csv", encoding='utf-8', index=False)

In [ ]:
#pd.read_csv("StemmedWord2vecTop3.csv")